In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import sklearn as skl
import tensorflow as tf

In [2]:
# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
#Generate cat var list
charitycat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

#Check unique values
charity_df[charitycat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [4]:
#Drop columns
charity_df = charity_df.drop(columns=["NAME", "EIN", "CLASSIFICATION"], axis = 1)
charity_df.head()

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
#Rename APPLICATION column
charity_df = charity_df.rename(columns ={"APPLICATION_TYPE": "Application"})
charity_df.head()

,Application,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
charitycat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

charity_df[charitycat].nunique()

Application               17
AFFILIATION                6
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [7]:
charity_df.Application.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: Application, dtype: int64

In [8]:
#print count for binning
app_counts = charity_df.Application.value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: Application, dtype: int64

In [9]:
# determine which values to replace
replace_app = list(app_counts[app_counts < 1000].index)

# replace in df
for application in replace_app:
    charity_df.Application = charity_df.Application.replace(application,"Other")


# check to make sure binning was successful
charity_df.Application.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: Application, dtype: int64

In [10]:
charity_df[charitycat].nunique()

Application               6
AFFILIATION               6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [11]:
# OHE instance
enc = OneHotEncoder(sparse=False)

# fit and transform the OneHotEncoder using the cat var list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charitycat]))

# add the encoded var names to the df
encode_df.columns = enc.get_feature_names(charitycat)
encode_df.head()

,Application_Other,Application_T19,Application_T3,Application_T4,Application_T5,Application_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charitycat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,Application_Other,Application_T19,Application_T3,Application_T4,Application_T5,Application_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop(["IS_SUCCESSFUL"],1).values

# sklearn to split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [14]:
X_scaler = skl.preprocessing.StandardScaler()

X_scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

In [16]:
# add the input and hidden layer
number_inputs = 34
number_hidden_nodes = 6

nn_model.add(tf.keras.layers.Dense(units=number_hidden_nodes, activation="tanh", input_dim=number_inputs))

# add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [17]:
# check the structure of the Sequential model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 210       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 217
Trainable params: 217
Non-trainable params: 0
_________________________________________________________________


In [18]:
# compile
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=125)

Epoch 1/125
804/804 [==============================] - 0s 406us/step - loss: 0.5988 - accuracy: 0.7061
Epoch 2/125
804/804 [==============================] - 0s 446us/step - loss: 0.5797 - accuracy: 0.7247
Epoch 3/125
804/804 [==============================] - 0s 421us/step - loss: 0.5778 - accuracy: 0.7257
Epoch 4/125
804/804 [==============================] - 0s 415us/step - loss: 0.5764 - accuracy: 0.7257
Epoch 5/125
804/804 [==============================] - 0s 410us/step - loss: 0.5755 - accuracy: 0.7259
Epoch 6/125
804/804 [==============================] - 0s 406us/step - loss: 0.5746 - accuracy: 0.7264
Epoch 7/125
804/804 [==============================] - 0s 405us/step - loss: 0.5741 - accuracy: 0.7260
Epoch 8/125
804/804 [==============================] - 0s 407us/step - loss: 0.5734 - accuracy: 0.7265
Epoch 9/125
804/804 [==============================] - 0s 403us/step - loss: 0.5730 - accuracy: 0.7268
Epoch 10/125
804/804 [==============================] - 0s 407us/step - l

804/804 [==============================] - 0s 410us/step - loss: 0.5681 - accuracy: 0.7286
Epoch 80/125
804/804 [==============================] - 0s 410us/step - loss: 0.5681 - accuracy: 0.7287
Epoch 81/125
804/804 [==============================] - 0s 403us/step - loss: 0.5681 - accuracy: 0.7289
Epoch 82/125
804/804 [==============================] - 0s 420us/step - loss: 0.5679 - accuracy: 0.7283
Epoch 83/125
804/804 [==============================] - 0s 423us/step - loss: 0.5679 - accuracy: 0.7288
Epoch 84/125
804/804 [==============================] - 0s 408us/step - loss: 0.5680 - accuracy: 0.7287
Epoch 85/125
804/804 [==============================] - 0s 415us/step - loss: 0.5678 - accuracy: 0.7284
Epoch 86/125
804/804 [==============================] - 0s 408us/step - loss: 0.5678 - accuracy: 0.7287
Epoch 87/125
804/804 [==============================] - 0s 402us/step - loss: 0.5680 - accuracy: 0.7287
Epoch 88/125
804/804 [==============================] - 0s 417us/step - loss:

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5733 - accuracy: 0.7223
Loss: 0.5733250975608826, Accuracy: 0.7223323583602905
